# Convert SQLite outputs to parquet files with cytotable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.cellprofiler_processing/CellProfiler_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 3 plates in this dataset. Below are the names:
localhost230405150001
localhost220512140003_KK22-05-198
localhost220513100001_KK22-05-198_FactinAdjusted


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet")
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000
    )
    
print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: localhost230405150001
Starting conversion with cytotable for plate: localhost220512140003_KK22-05-198
Starting conversion with cytotable for plate: localhost220513100001_KK22-05-198_FactinAdjusted
All plates have been converted with cytotable!


# Load in converted profiles to update

In [7]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)
    
    # Drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])
    
    # Columns to move to the front
    columns_to_move = ['Cells_Location_Center_X', 'Cells_Location_Center_Y', "Image_Count_Cells"]
    
    # Rearrange columns and add "Metadata" prefix in one line
    df = (df[columns_to_move + [col for col in df.columns if col not in columns_to_move]]
                  .rename(columns=lambda col: 'Metadata_' + col if col in columns_to_move else col))
    
    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)


## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [8]:
converted_df = pd.read_parquet("./data/converted_profiles/localhost220512140003_KK22-05-198_converted.parquet")

print(converted_df.shape)
converted_df.head()

(43204, 2016)


,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,509.949487,25.078927,35,1.0,localhost220512140003_KK22-05-198,f00,A01,1.0,1.0,1.0,...,6.485036,6.564180,0.570463,0.480469,0.500099,0.541887,0.910776,0.809389,0.825054,0.930508
1,238.995447,104.179514,35,1.0,localhost220512140003_KK22-05-198,f00,A01,2.0,2.0,4.0,...,4.623373,4.612059,0.668536,0.625061,0.584534,0.571266,0.378195,0.383930,0.377923,0.372965
2,199.191002,189.827676,35,1.0,localhost220512140003_KK22-05-198,f00,A01,4.0,4.0,7.0,...,3.148509,2.988588,0.461019,0.414894,0.420354,0.427692,0.697642,0.668938,0.692478,0.723110
3,363.699578,247.405714,35,1.0,localhost220512140003_KK22-05-198,f00,A01,7.0,7.0,10.0,...,7.012669,6.914432,7.268538,8.047236,8.035632,6.831862,5.814823,6.037612,5.872898,5.561801
4,550.957839,243.532560,35,1.0,localhost220512140003_KK22-05-198,f00,A01,8.0,8.0,11.0,...,4.043299,4.017283,0.511564,0.499693,0.505643,0.516494,0.384940,0.375795,0.386942,0.381264
